# EEG data classification Guinnea Bissau

This notebook contains experiments with an EEG dataset. The classes are Epilepsy: 0 or Control 1.

Load dependences and setting output configuration

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
%pylab inline
%load_ext autoreload
%autoreload 2

Using Theano backend.


Populating the interactive namespace from numpy and matplotlib


## Load data from npy files

Specify location of npy files:

In [2]:
datapath = '/media/windows-share/EEGs_Guinea-Bissau_np/'
#datapath = '/media/sf_VBox_Shared/timeseries/EEGs_Guinea-Bissau_np/'#

Load data stored in 10 seconds at 128 Hertz corresponding to the experiment where the participant had the eyes closed:

In [3]:
condition = '_10seconds_closed.npy'
X_train = np.load(datapath+'X_train'+condition)
y_train = np.load(datapath+'y_train'+condition)
X_val = np.load(datapath+'X_valid'+condition)
y_val = np.load(datapath+'y_valid'+condition)
X_test = np.load(datapath+'X_test'+condition)
y_test = np.load(datapath+'y_test'+condition)

In [4]:
classlabels = list(set(y_train))
mapclasses = {classlabels[i] : i for i in range(len(classlabels))}
print(mapclasses)

{'Epilepsy': 0, 'Control': 1}


In [5]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_val], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [6]:
y_val_binary

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [7]:
X_train.shape

(108, 1280, 14)

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
np.random.seed(1234)
from keras.layers import Dense, Activation, Convolution1D, Lambda, \
    Convolution2D, Flatten, \
    Reshape, LSTM, Dropout, TimeDistributed, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam

In [9]:
x_shape = X_train.shape
class_number = y_train_binary.shape[1]
dim_length = x_shape[1]  # number of samples in a time series
dim_channels = x_shape[2]  # number of channels
outputdim = class_number  # number of classes

In [12]:
filters = [50,50,40]
fc_hidden_nodes = 3
learning_rate=0.01
regularization_rate=0.01
weightinit = 'lecun_uniform' 

In [13]:
model = Sequential()
model.add(BatchNormalization(
            input_shape=(dim_length, dim_channels),
            mode=0, axis=2))
for filter_number in filters:
    model.add(Convolution1D(filter_number, 3, border_mode='same',
                               W_regularizer=l2(regularization_rate),
                                init=weightinit))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

In [14]:
model.add(Flatten())
model.add(Dense(output_dim=fc_hidden_nodes,
                W_regularizer=l2(regularization_rate),
                init=weightinit))  # Fully connected layer
model.add(Activation('relu'))  # Relu activation
model.add(Dense(output_dim=outputdim, init=weightinit))
model.add(BatchNormalization())
model.add(Activation("softmax"))  # Final classification layer

In [15]:
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])

In [18]:
y_train.shape

(108,)

In [26]:
subset_size = 60
X_train_sub = X_train[:subset_size, :, :]
y_train_sub = y_train_binary[:subset_size, :]
nr_epochs = 10
verbose = True

In [27]:
history = model.fit(X_train_sub, y_train_sub,
                            nb_epoch=nr_epochs, batch_size=20,
                            # see comment on subsize_set
                            validation_data=(X_val, y_val_binary),
                            verbose=verbose)

Train on 60 samples, validate on 20 samples
Epoch 1/10
60/60 [==============================] - 2s - loss: 3.5992 - acc: 0.8333 - val_loss: 1.3754 - val_acc: 0.5500
Epoch 2/10
60/60 [==============================] - 2s - loss: 3.7174 - acc: 0.7333 - val_loss: 1.0708 - val_acc: 0.5000
Epoch 3/10
60/60 [==============================] - 2s - loss: 3.7202 - acc: 0.8000 - val_loss: 0.7497 - val_acc: 0.4500
Epoch 4/10
60/60 [==============================] - 2s - loss: 3.5478 - acc: 0.9167 - val_loss: 1.1704 - val_acc: 0.5000
Epoch 5/10
60/60 [==============================] - 2s - loss: 3.4394 - acc: 0.9167 - val_loss: 0.8118 - val_acc: 0.6500
Epoch 6/10
60/60 [==============================] - 2s - loss: 3.2796 - acc: 0.9500 - val_loss: 0.8080 - val_acc: 0.7000
Epoch 7/10
60/60 [==============================] - 2s - loss: 3.1636 - acc: 0.9167 - val_loss: 0.7712 - val_acc: 0.7000
Epoch 8/10
60/60 [==============================] - 2s - loss: 2.9479 - acc: 0.9667 - val_loss: 0.8620 - val_

In [25]:
history.history

{'acc': [0.46666665871938068, 0.55000001192092896, 0.58333333333333337, 0.75],
 'loss': [2.3554260730743408,
  2.9451981385548911,
  3.3592026233673096,
  3.4458752473195395],
 'val_acc': [0.44999998807907104,
  0.44999998807907104,
  0.44999998807907104,
  0.44999998807907104],
 'val_loss': [8.8649530410766602,
  8.8649530410766602,
  8.8649530410766602,
  8.5380735397338867]}

https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent